# Plan

- 'maximum_speed_limit' 이 랜덤포레스트에서 높은 중요도를 가짐 : 주형 EDA
- test 에 속도 40 제한 구역이 없다.
- 제한속도별로 랜덤포레스트를 돌린후 다시합치는 계획

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
train = train.drop(['base_date','road_in_use','vehicle_restricted','height_restricted'],axis=1)
test = test.drop(['base_date','road_in_use','vehicle_restricted','height_restricted'],axis=1)

In [12]:
train["maximum_speed_limit"].value_counts()

60.0    1665573
50.0    1103682
70.0     995077
80.0     700334
30.0     229761
40.0       6790
Name: maximum_speed_limit, dtype: int64

In [54]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
train = train.drop(['base_date','road_in_use','vehicle_restricted','height_restricted'],axis=1)
test = test.drop(['base_date','road_in_use','vehicle_restricted','height_restricted'],axis=1)

from haversine import haversine, Unit, haversine_vector
start = [(i-90,j) for i,j in zip(train['start_longitude'],train['start_latitude'])]
end = [(i-90,j) for i,j in zip(train['end_longitude'],train['end_latitude'])]
distance = [haversine(i, j, unit=Unit.KILOMETERS) for i,j in zip(start,end)]

# train = train.drop(['start_longitude','end_longitude','start_latitude','end_latitude'],axis=1)
train['distance'] = distance

from haversine import haversine, Unit, haversine_vector
start = [(i-90,j) for i,j in zip(test['start_longitude'],test['start_latitude'])]
end = [(i-90,j) for i,j in zip(test['end_longitude'],test['end_latitude'])]
distance = [haversine(i, j, unit=Unit.KILOMETERS) for i,j in zip(start,end)]

# train = train.drop(['start_longitude','end_longitude','start_latitude','end_latitude'],axis=1)
test['distance'] = distance

- 제한속도별로 랜덤포레스트를 돌린후 다시합치는 계획 : 예시로 속도 80제한 데이터만 추출하여 랜덤포레스트 돌려본다.
- "maximum_speed_limit" 분리되면 사용되지 않으므로 레이블인코딩 제외

In [55]:
from sklearn.preprocessing import LabelEncoder
str_col = ["base_hour","day_of_week","lane_count","road_rating","multi_linked","connect_code"
           ,"weight_restricted","road_type","start_turn_restricted",
          "end_turn_restricted","road_name","start_node_name","end_node_name",
          'distance'] # "maximum_speed_limit"
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [5]:
# train = train[train["maximum_speed_limit"]==80]
# y_train = train['target'] 
# X_train = train.drop(['id', 'target'], axis=1)

# test = test[test["maximum_speed_limit"]==80]
# test_id = test['id']
# test = test.drop(['id'], axis=1)

# print(X_train.shape)
# print(y_train.shape)
# print(test.shape)

(700334, 20)
(700334,)
(9614, 20)


In [43]:
# prediction_table = pd.DataFrame(test_id)

In [6]:
# X_train,X_test,y_train,y_test = train_test_split(
#     X_train,y_train,test_size=0.04,random_state=10
# )

In [7]:
# from sklearn.ensemble import RandomForestRegressor
# tp = RandomForestRegressor(random_state=56)
# tp.fit(X_train,y_train)
# pred = tp.predict(X_test)
# mae = mean_absolute_error(pred,y_test)
# print(mae)

2.4394020846719497


In [46]:
# prediction_table['target'] = tp.predict(test)

In [47]:
# prediction_table.to_csv('./target80.csv')

In [ ]:
# from sklearn.ensemble import VotingRegressor
# from sklearn.ensemble import RandomForestRegressor


# r1 = RandomForestRegressor(random_state=10,criterion='absolute_error')
# r2 = CatBoostRegressor(random_state=10)
# r3 = ExtraTreesRegressor(random_state=10,criterion='absolute_error')
# vote = VotingRegressor([('r1', r1), ('r2', r2), ('r3', r3)])
# vote.fit(X_train,y_train)

In [8]:
# train = train.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)


In [56]:
train = train[train['connect_code'] == 0]
train = train[train['multi_linked'] == 0]
train = train[train['maximum_speed_limit'] != 40]

In [51]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

def rf_speed(speed,train,test):
#     train = pd.read_parquet('./train.parquet')
#     test = pd.read_parquet('./test.parquet')
#     train = train.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)
#     test = test.drop(['road_in_use','vehicle_restricted','height_restricted'],axis=1)

#     str_col = ["base_date","base_hour","day_of_week","lane_count","road_rating","multi_linked","connect_code"
#            ,"weight_restricted","road_type","start_latitude","start_longitude","start_turn_restricted",
#           "end_turn_restricted","road_name","start_node_name","end_node_name",'end_latitude','end_longitude','distance'] # "maximum_speed_limit"
#     for i in str_col:
#         le = LabelEncoder()
#         le=le.fit(train[i])
#         train[i]=le.transform(train[i])

#         for label in np.unique(test[i]):
#             if label not in le.classes_: 
#                 le.classes_ = np.append(le.classes_, label)
#         test[i]=le.transform(test[i])

    train = train[train["maximum_speed_limit"]==speed]
    y_train = train['target'] 
    X_train = train.drop(['id', 'target'], axis=1)

    test = test[test["maximum_speed_limit"]==speed]
    test_id = test['id']
    test = test.drop(['id'], axis=1)

    prediction_table = pd.DataFrame(test_id)

    X_train,X_test,y_train,y_test = train_test_split(
        X_train,y_train,test_size=0.04,random_state=10
    )

    r1 = RandomForestRegressor(random_state=56)
    # r2 = CatBoostRegressor(random_state=10)
#     r3 = ExtraTreesRegressor(random_state=56)
#    r4 = lgb.LGBMRegressor(random_state=56,error_score=True)
#    vote = VotingRegressor([('r1', r1), ('r3', r3)])

    parms = {'verbose':[True]}
    grid_cv = GridSearchCV(r1,param_grid=parms,cv=6,n_jobs=-1,scoring='neg_mean_absolute_error')# ,param_grid=parms
    grid_cv.fit(X_train,y_train)

    pred = grid_cv.predict(X_test)
    mae = mean_absolute_error(pred,y_test)
    print(mae)

    prediction_table['target'] = grid_cv.predict(test)
    prediction_table.to_csv(f'./target{speed}.csv')
    print(f'save {speed}')

In [52]:
rf_speed(80,train,test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


3.187297536092579
save 80


In [53]:
rf_speed(70,train,test)
rf_speed(60,train,test)
rf_speed(50,train,test)
rf_speed(30,train,test) # voting 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


3.755884298990132


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished


save 70


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  6.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished


save 50


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   22.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


2.895622668667186
save 30


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [ ]:
rf_speed(50,train,test)

In [23]:
ans = pd.concat([pd.read_csv('target30.csv'),pd.read_csv('target50.csv'),
          pd.read_csv('target60.csv'),pd.read_csv('target70.csv'),
          pd.read_csv('target80.csv')]).drop('Unnamed: 0',axis=1)
ans = ans.sort_values('id')
ans = ans.set_index('id')
ans.to_csv('./submit.csv')